In [1]:
import os
import requests
from subprocess import *
from bs4 import BeautifulSoup
import json
import base64

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

#EDA

In [4]:
# Data= pd.read_csv('/content/drive/MyDrive/Phishing Dataset/dataset_full.csv')

In [5]:
# Data.describe()

In [6]:
# Data.isnull().values.sum()

In [7]:
# Data.isnull().sum()

In [8]:
# Data.head()

In [9]:
# #Lets see a the distribution of Length of URL Data
# sns.set(rc={'figure.figsize':(15,5)})
# fig = plt.figure()
# sns.histplot(data=Data, x='length_url', bins=100)
# plt.axvline(Data['length_url'].mean(), c='red', ls='-', lw=3, label='Mean Length')
# plt.axvline(Data['length_url'].median(),c='blue',ls='-',lw=3, label='Median Length')
# plt.title('Distribution of Length', fontsize=20, fontweight='bold')
# plt.legend()
# plt.show()


In [10]:
# print("Highest allowed Length",Data['length_url'].mean() + 3*Data['length_url'].std())
# print("Lowest allowed Length",Data['length_url'].mean() - 3*Data['length_url'].std())

In [11]:
# Data_n = Data[ (Data['length_url'] < 200)]

In [12]:
#Lets see a the distribution of Length of URL Data
# sns.set(rc={'figure.figsize':(15,5)})
# fig = plt.figure()
# sns.histplot(data=Data_n, x='length_url', bins=100)
# plt.axvline(Data_n['length_url'].mean(), c='red', ls='-', lw=3, label='Mean Length')
# plt.axvline(Data_n['length_url'].median(),c='blue',ls='-',lw=3, label='Median Length')
# plt.title('Distribution of Length', fontsize=20, fontweight='bold')
# plt.legend()
# plt.show()

In [13]:
# plt.scatter(Data.length_url, Data.phishing)

In [14]:
# #Let's start with just one variable to demonstrate
# fig, ax = plt.subplots(1,2)
# sns.boxplot(data=Data_n, x='phishing', y='qty_ip_resolved', ax=ax[0])
# sns.histplot(Data_n, x="qty_ip_resolved", hue="phishing", kde=True, ax=ax[1])
# plt.suptitle("phishing", fontsize=20, fontweight='bold')
# fig.show()

In [15]:
# sns.countplot(x ='phishing', hue = "qty_ip_resolved", data = Data)

In [16]:
# Data.drop('url_len_range',
#   axis='columns', inplace = True)

In [17]:
# url_len_range = pd.cut(Data.length_url,bins=[0,20,40,60,80,100,4000])
# Data.insert(5,'url_len_range',url_len_range)

In [18]:
# sns.countplot(x ='phishing', hue = "url_len_range", data = Data)

In [19]:
# Data.boxplot('length_url','phishing',rot = 0,figsize=(5,6))

In [20]:
# Data.boxplot('qty_at_url','phishing',rot = 0,figsize=(5,6))

In [21]:
# sns.countplot(x ='phishing', hue = "qty_at_url", data = Data)

In [22]:
# Data['phishing'].value_counts()

In [23]:
# likely_cat = {}
# for var in Data.iloc[:,1:].columns:
#     likely_cat[var] = 1.*Data[var].nunique()/Data[var].count() < 0.002 

# num_cols = []
# cat_cols = []
# for col in likely_cat.keys():
#     if (likely_cat[col] == False):
#         num_cols.append(col)
#     else:
#         cat_cols.append(col)

In [24]:
# #Taking all columns except URL 
# corr = Data[num_cols].corr()

# fig = plt.figure(figsize=(12,12),dpi=80)
# mask = np.triu(np.ones_like(corr, dtype=bool))
# sns.heatmap(corr, mask=mask, cmap='BuPu', robust=True, center=0,
#             square=True, linewidths=.5)
# plt.title('Correlation of Numerical(Continous) Features', fontsize=15)
# plt.show()

In [25]:
# df_distr =Data.groupby('phishing')[num_cols].mean().reset_index().T
# df_distr.rename(columns={0:'0_Label',1:"1_Label"}, inplace=True)

# #plt.style.use('ggplot')
# plt.rcParams['axes.facecolor']='w'
# ax = df_distr[1:-3][['0_Label','1_Label']].plot(kind='bar', title ="Distribution of Average values across Target", figsize=(12, 8), legend=True, fontsize=12)
# ax.set_xlabel("Numerical Features", fontsize=14)
# ax.set_ylabel("Average Values", fontsize=14)
# #ax.set_ylim(0,500000)
# plt.show()

In [26]:
# !pip install python-whois

# Feature Extraction

In [27]:
!pip install whois

  Created wheel for whois: filename=whois-0.9.13-py3-none-any.whl size=10090 sha256=ebdd95b126132fea1c0305c159dbd1d9307ae23773f1e08ede2227a7b68209e6
  Stored in directory: c:\users\asodh\appdata\local\pip\cache\wheels\e5\05\66\bb7e2a3840045aad2b1c08e7da85f471ac86514f000cf93f4f
Successfully built whois


In [28]:
from urllib.parse import urlparse
# import favicon
import xml.etree.ElementTree as ET 
# import tldextract
import datetime
from dateutil.relativedelta import relativedelta
import whois

In [29]:
Normal_url ='http://intego3.info/EXEL/index.php'
phish_url = 'https://www.computerhope.com/issues/ch000254.htm'

1. Check IP and Hexa

In [30]:
def to_find_having_ip_add(url):
  import string
  index = url.find("://")
  split_url = url[index+3:]
  # print(split_url)
  index = split_url.find("/")
  split_url = split_url[:index]
  # print(split_url)
  split_url = split_url.replace(".", "")
  # print(split_url)
  counter_hex = 0
  for i in split_url:
    if i in string.hexdigits:
      counter_hex +=1

  total_len = len(split_url)
  having_IP_Address = 1
  if counter_hex >= total_len:
    having_IP_Address = -1

  return having_IP_Address

In [31]:
print(to_find_having_ip_add(Normal_url))
print(to_find_having_ip_add(phish_url))

1
1


2. URL_len

In [32]:
def to_find_url_len(url):
  URL_Length = 1
  # print(len(url))
  if len(url)>=75:
    URL_Length = -1
  elif len(url)>=54 and len(url)<=74:
    URL_length = 0
  
  return URL_Length

In [33]:
print(to_find_url_len(Normal_url))
print(to_find_url_len(phish_url))

1
1


3. Shortened URL extraction

In [34]:
def get_complete_URL(shortened_url):
  command_stdout = Popen(['curl', shortened_url], stdout=PIPE).communicate()[0]
  output = command_stdout.decode('utf-8')
  href_index = output.find("href=")
  if href_index == -1:
    href_index = output.find("HREF=")
  splitted_ = output[href_index:].split('"')
  expanded_url = splitted_[1]
  return expanded_url


In [35]:
  # command_stdout = Popen(['curl', shortened_url], stdout=PIPE).communicate()[0]
  # output = command_stdout.decode('utf-8')
  # href_index = output.find("href=")
  # if href_index == -1:
  #   href_index = output.find("HREF=")
  # splitted_ = output[href_index:].split('"')
  # expanded_url = splitted_[1]
  # return expanded_url

In [36]:
# print(get_complete_URL(Normal_url))
# print(get_complete_URL(phish_url))

In [37]:

def check_for_shortened_url(url):
  famous_short_urls = ["bit.ly", "tinyurl.com", "goo.gl",
                       "rebrand.ly", "t.co", "youtu.be",
                       "ow.ly", "w.wiki", "is.gd"]

  # domain_of_url = url.split("://")[1]
  domain_of_url = url.split("/")[0]
  status = 1
  if domain_of_url in famous_short_urls:
    status = -1

  complete_url = None
  if status == -1:
    complete_url = get_complete_URL(url)

  return (status, complete_url)

In [38]:
check_for_shortened_url(phish_url)

(1, None)

In [39]:
check_for_shortened_url(Normal_url)

(1, None)

4. @ in URL

In [40]:
def to_find_at(url):
  label = 1
  index = url.find("@")
  if index!=-1:
    label = -1
  
  return label

In [41]:
print(to_find_at(Normal_url))
print(to_find_at(phish_url))

1
1


5. Redirect

In [42]:
def to_find_redirect(url):
  index = url.find("://")
  split_url = url[index+3:]
  label = 1  
  index = split_url.find("//")
  if index!=-1:
    label = -1
  
  return label

In [43]:
print(to_find_redirect(Normal_url))
print(to_find_redirect(phish_url))

1
1


6. "-" in domain (Prefix_suffix)

In [44]:
def to_find_prefix(url):
  index = url.find("://")
  split_url = url[index+3:]
  # print(split_url)
  index = split_url.find("/")
  split_url = split_url[:index]
  # print(split_url)
  label = 1
  index = split_url.find("-")
  # print(index)
  if index!=-1:
    label = -1
  
  return label

In [45]:
print(to_find_prefix(Normal_url))
print(to_find_prefix(phish_url))

1
1


7. Multi-domains presence

In [46]:
def to_find_multi_domains(url):
  url = url.split("://")[1]
  url = url.split("/")[0]
  index = url.find("www.")
  split_url = url
  if index!=-1:
    split_url = url[index+4:]
  # print(split_url)
  index = split_url.rfind(".")
  # print(index)
  if index!=-1:
    split_url = split_url[:index]
  # print(split_url)
  counter = 0
  for i in split_url:
    if i==".":
      counter+=1
  
  label = 1
  if counter==2:
    label = 0
  elif counter >=3:
    label = -1
  
  return label

In [47]:
print(to_find_multi_domains(Normal_url))
print(to_find_multi_domains(phish_url))

1
1


8. Authority

In [2]:
def to_find_authority(url):
  index_https = url.find("https://")
  valid_auth = ["GeoTrust", "GoDaddy", "Network Solutions", "Thawte", "Comodo", "Doster" , "VeriSign", "LinkedIn", "Sectigo",
                "Symantec", "DigiCert", "Network Solutions", "RapidSSLonline", "SSL.com", "Entrust Datacard", "Google", "Facebook"]
  
  cmd = "curl -vvI " + url

  stdout = Popen(cmd, shell=True, stderr=PIPE, env={} )
  # print(stdout)
  output = stdout.read()
  # print(output)
  std_out = output.decode('UTF-8',errors="ignore" )
  # print(std_out)
  index = std_out.find("O=")

  split = std_out[index+2:]
  index_sp = split.find(" ")
  cur = split[:index_sp]
  
  index_sp = cur.find(",")
  if index_sp!=-1:
    cur = cur[:index_sp]
  # print(cur)
  label = -1
  if cur in valid_auth and index_https!=-1:
    label = 1
  
  return label

In [54]:
pip install py2exe


  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=16d6f66d642fa92bc50673aed194c98cfc4ba61c35916a1859ae5b3e1f32769b
  Stored in directory: c:\users\asodh\appdata\local\pip\cache\wheels\05\42\e9\7d0273e82936b525e10e4080595cd0aad0b3481d78b5a3aecc
Successfully built pefile


In [4]:
import subprocess
subprocess.Popen(cmd, stderr=subprocess.STDOUT, stdout=subprocess.PIPE)

NameError: name 'cmd' is not defined

In [3]:
print(to_find_authority(r'https://www.weibull.com/hotwire/issue88/relbasics88.htm'))

NameError: name 'Popen' is not defined

In [ ]:
print(to_find_authority(Normal_url))
# print(to_find_authority(phish_url))

9. Domain registration Length

In [ ]:
!pip install tldextract

In [ ]:
import tldextract

In [ ]:
def dregisterlen(u):
    extract_res = tldextract.extract(u)
    ul = extract_res.domain + "." + extract_res.suffix
    try:
        wres = whois.whois(u)
        f = wres["Creation Date"][0]
        s = wres["Registry Expiry Date"][0]
        if(s>f+relativedelta(months=+12)):
            return 1
        else:
            return -1
    except:
        return -1

In [ ]:
print(dregisterlen(Normal_url))
print(dregisterlen(phish_url))

10. Favicon

In [ ]:
!pip install favicon

In [ ]:
import requests
import favicon

In [ ]:
def check_favicon(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain

  favs = favicon.get(url)
  # print(favs)
  match = 0
  for favi in favs:
    url2 = favi.url
    extract_res = tldextract.extract(url2)
    url_ref2 = extract_res.domain

    if url_ref in url_ref2:
      match += 1

  if match >= len(favs)/2:
    return 1
  return -1

In [ ]:
url = "https://www.iiitd.ac.in"
check_favicon(url)

11. https as part of domain

In [ ]:
def existenceoftoken(u):
    # Assumption - pagename cannot start with this token
    ix = u.find("//https")
    # print(ix)
    if(ix==-1):
        return 1
    else:
        return -1

In [ ]:
print(existenceoftoken(Normal_url))
print(existenceoftoken(phish_url))

## Abnormal Based Features

1. Request URL

In [ ]:
def check_request_URL(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain

  command_stdout = Popen(['curl', 'https://api.hackertarget.com/pagelinks/?q=' + url], stdout=PIPE).communicate()[0]
  links = command_stdout.decode('utf-8').split("\n")

  count = 0

  for link in links:
    extract_res = tldextract.extract(link)
    url_ref2 = extract_res.domain

    if url_ref not in url_ref2:
      count += 1

  count /= len(links)

  if count < 0.22:
    return 1
  elif count < 0.61:
    return 0
  else:
    return -1

In [ ]:
url = "https://xavier-net.gq/?login=do"

In [ ]:
check_request_URL(url)

In [ ]:
print(check_request_URL(Normal_url))
print(check_request_URL(phish_url))

2. URL of Anchor

In [ ]:
def check_URL_of_anchor(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain
  # remove_last_underscore(url)
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, "lxml")
  a_tags = soup.find_all('a')

  if len(a_tags) == 0:
    return 1

  invalid = ['#', '#content', '#skip', 'JavaScript::void(0)']
  bad_count = 0
  for t in a_tags:
    link = t['href']

    if link in invalid:
      bad_count += 1

    if url_validator(link):
      extract_res = tldextract.extract(link)
      url_ref2 = extract_res.domain

      if url_ref not in url_ref2:
        bad_count += 1

  bad_count /= len(a_tags)

  if bad_count < 0.31:
    return 1
  elif bad_count <= 0.67:
    return 0
  return -1

In [ ]:
def url_validator(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc, result.path])
    except:
        return False

In [ ]:
print(check_URL_of_anchor(Normal_url))
print(check_URL_of_anchor(phish_url))

3. Tags

In [ ]:
def tags(u):
    programhtml = requests.get(u).text
    s = BeautifulSoup(programhtml,"lxml")
    mtags = s.find_all('Meta')
    ud = tldextract.extract(u)
    upage = ud.domain
    mcount = 0
    for i in mtags:
        u1 = i['href']
        currpage = tldextract.extract(u1)
        u1page = currpage.domain
        if currpage not in ulpage:
            mcount+=1
    scount = 0
    stags = s.find_all('Script')
    for j in stags:
        u1 = j['href']
        currpage = tldextract.extract(u1)
        u1page = currpage.domain
        if currpage not in u1page:
            scount+=1
    lcount = 0
    ltags = s.find_all('Link')
    for k in ltags:
        u1 = k['href']
        currpage = tldextract.extract(u1)
        u1page = currpage.domain
        if currpage not in u1page:
            lcount+=1
    percmtag = 0
    percstag = 0
    percltag = 0

    if len(mtags) != 0:
      percmtag = (mcount*100)//len(mtags)
    if len(stags) != 0:
      percstag = (scount*100)//len(stags)
    if len(ltags) != 0:
      percltag = (lcount*100)//len(ltags)
      
    if(percmtag+percstag+percltag<17):
        return 1
    elif(percmtag+percstag+percltag<=81):
        return 0
    return -1

In [ ]:
print(tags(Normal_url))
print(tags(phish_url))

4. SFH

In [ ]:
def sfh(u):
    programhtml = requests.get(u).text
    s = BeautifulSoup(programhtml,"lxml")
    try:
        f = str(s.form)
        ac = f.find("action")
        if(ac!=-1):
            i1 = f[ac:].find(">")
            u1 = f[ac+8:i1-1]
            if(u1=="" or u1=="about:blank"):
                return -1
            er1 = tldextract.extract(u)
            upage = erl.domain
            erl2 = tldextract.extract(u1)
            usfh = erl2.domain
            if upage in usfh:
                return 1
            return 0
        else:
            # Check this point
            return 1
    except:
        # Check this point
        return 1

In [ ]:
print(sfh(Normal_url))
print(sfh(phish_url))

5. Submitting to Email

In [ ]:
def check_submit_to_email(url):
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, "lxml")
  # Check if no form tag
  form_opt = str(soup.form)
  idx = form_opt.find("mail()")
  if idx == -1:
    idx = form_opt.find("mailto:")

  if idx == -1:
    return 1
  return -1

In [ ]:
print(check_submit_to_email(Normal_url))
print(check_submit_to_email(phish_url))

## HTML and JavaScript based Features

1. Redirect

In [ ]:
def redirect(url):
  opt = Popen(["sh", "/red.sh", url], stdout=PIPE).communicate()[0]
  opt = opt.decode('utf-8')
  # print(opt)
  opt = opt.split("\n")
  
  new = []
  for i in opt:
    i = i.replace("\r", " ")
    new.extend(i.split(" "))
  

  count = 0
  for i in new:
   
    if i.isdigit():
      conv = int(i)
      if conv > 300 and conv<310:
        count += 1

  last_url = None
  for i in new[::-1]:
    if url_validator(i):
      last_url = i
      break

  if (count<=1):
    return 1, last_url
  elif count>=2 and count <4:
    return 0, last_url
  return -1, last_url

In [ ]:
ur = "https://bit.ly/segfault"

In [ ]:
phish_url2 = "https://oxify.me/tuT2y"

In [ ]:
redirect(ur)

In [ ]:
print(redirect(Normal_url))
print(redirect(phish_url))

2. On mouseover

In [ ]:
def check_onmouseover(url):
  try:
    html_content = requests.get(url).text
  except:
    return -1
  soup = BeautifulSoup(html_content, "lxml")
  if str(soup).lower().find('onmouseover="window.status') != -1:
    return -1
  return 1

In [ ]:
url = "https://google.com"
check_onmouseover(url)

In [ ]:
print(check_onmouseover(Normal_url))
print(check_onmouseover(phish_url))

3. Rightclick

In [ ]:
def check_rightclick(url):
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, "lxml")
  if str(soup).lower().find("preventdefault()") != -1:
    return -1
  elif str(soup).lower().find("event.button==2") != -1:
    return -1
  elif str(soup).lower().find("event.button == 2") != -1:
    return -1
  return 1

In [ ]:
check_rightclick(Normal_url)

In [ ]:
check_rightclick(phish_url)

4. IFrame

In [ ]:
def check_iframe(url):
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, "lxml")
  if str(soup.iframe).lower().find("frameborder") == -1:
    return 1
  return -1

In [ ]:
check_iframe("https://www.google.com")

## Domain based Features

1. Age of Domain

In [ ]:
url = 'https://xavier-net.gq/?login=do'

In [ ]:
def check_age_of_domain(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain + "." + extract_res.suffix
  try:
    whois_res = whois.whois(url)
    if datetime.datetime.now() > whois_res["creation_date"][0] + relativedelta(months=+6):
      return 1
    else:
      return -1
  except:
    return -1

In [ ]:
check_age_of_domain(url)

In [ ]:
print(check_age_of_domain(Normal_url))
print(check_age_of_domain(phish_url))

2. DNS Record

In [ ]:
!apt-get install whois

In [ ]:
!pip install python-whois

In [ ]:
def check_dns_record(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain + "." + extract_res.suffix
  try:
    whois_res = whois.whois(url)
    return 1
  except:
    return -1

In [ ]:
url = "goiigle.com"
check_dns_record(url)

In [ ]:
print(check_dns_record(Normal_url))
print(check_dns_record(phish_url))

3. Web traffic

In [ ]:
def check_web_traffic(url):
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain + "." + extract_res.suffix
  html_content = requests.get("https://www.alexa.com/siteinfo/" + url_ref).text
  soup = BeautifulSoup(html_content, "lxml")
  value = str(soup.find('div', {'class': "rankmini-rank"}))[42:].split("\n")[0].replace(",", "")

  if not value.isdigit():
    return -1

  value = int(value)
  if value < 100000:
    return 1
  return 0

In [ ]:
url = "https://hokk-i.com"

In [ ]:
check_web_traffic(url)

In [ ]:
print(check_web_traffic(Normal_url))
print(check_web_traffic(phish_url))

4. PageRank

In [ ]:
!pip install tldextract

In [ ]:
import tldextract
import requests

In [ ]:
def get_pagerank(url):
  # pageRankApi = open('/pageRankApi').readline()[:-2]
  extract_res = tldextract.extract(url)
  url_ref = extract_res.domain + "." + extract_res.suffix
  # headers = {'API-OPR': pageRankApi}
  headers = {'API-OPR':'4sgws8wocsc0oskgs4gsoo8gos88sg44gsk4s484'}
  domain = url_ref
  req_url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
  request = requests.get(req_url, headers=headers)
  result = request.json()
  # print(result)
  value = result['response'][0]['page_rank_decimal']
  if type(value) == str:
    value = 0

  if value < 2:
    return -1
  return 1


In [ ]:
get_pagerank("https://linked.in")

In [ ]:
print(get_pagerank(Normal_url))
print(get_pagerank(phish_url))

5. Statistical Reports

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def check_statistical_report(url):
  # # phishTankKey = open('/phishTankKey.txt')
  # # phishTankKey = phishTankKey.readline()[:-1]

  # headers = {
  #       'format': 'json',
  #       'app_key': '',
  #       }

  # def get_url_with_ip(URI):
  #     """Returns url with added URI for request"""
  #     url = "http://checkurl.phishtank.com/checkurl/"
  #     new_check_bytes = URI.encode()
  #     base64_bytes = base64.b64encode(new_check_bytes)
  #     base64_new_check = base64_bytes.decode('ascii')
  #     url += base64_new_check
  #     return url

  # def send_the_request_to_phish_tank(url, headers):
  #     """This function sends a request."""
  #     response = requests.request("POST", url=url, headers=headers)
  #     return response

  # url = get_url_with_ip(url)
  # print(url)
  # r = send_the_request_to_phish_tank(url, headers)
  # print(r)
  # def parseXML(xmlfile): 

  #   root = ET.fromstring(xmlfile) 
  #   verified = False
  #   for item in root.iter('verified'): 
  #     if item.text == "true":
  #       verified = True
  #       break

  #   phishing = False
  #   if verified:
  #     for item in root.iter('valid'): 
  #       if item.text == "true":
  #         phishing = True
  #         break

  #   return phishing

  # inphTank = parseXML(r.text)
  # # print(r.text)

  # if inphTank:
  #   return -1
  return 1

In [ ]:
check_statistical_report("https://www.kaggle.com/akashkr/phishing-url-eda-and-modelling")
# check_statistical_report('https://espace-facturation-9c8a01.ingress-erytho.easywp.com/org/customer_center/customer-IDPP00C793/login.php')

In [ ]:
Normal_url ='http://intego3.info/EXEL/index.php'
phish_url = 'https://www.computerhope.com/issues/ch000254.htm'

In [ ]:
print(check_statistical_report(Normal_url))
print(check_statistical_report(phish_url))

6. Number of Links Pointing to Page

In [ ]:
import urllib.request as Ureq
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlparse
def task(url):
    # x=Ureq.urlopen(url)
    # print ("Total Size of the Web Page = ",len(x.read())," Bytes")
    url_p=urlparse(url)
    domain='{uri.scheme}://{uri.netloc}/'.format(uri=url_p)
    # print (domain)
    resp=requests.get(url)
    soup=bs(resp.text,'html.parser')
    r=0
    for link in soup.find_all('a'):
        temp=link.get('href')
        if temp is not None and domain in temp:
          # print(temp)
          r=r+1
        # print(temp)
        # r=r+1
    # print("Total links pointing to same domain = ",r)
    if r > 2:
      return 1
    return -1


In [ ]:
Normal_url ='http://intego3.info/EXEL/index.php'
phish_url = 'https://www.computerhope.com/issues/ch000254.htm'

In [ ]:
task('https://www.youtube.com/results?search_query=page+rank+implementaton+via+page+rank+API+using+python')
print(task(Normal_url))
print(task(phish_url))

5. Google Index

In [ ]:
def extract_features(url):
  features_extracted = [0]*27
  # phStatus, expanded = check_for_shortened_url(url)
  # features_extracted[2] = phStatus
  # phStatus, last_url = redirect(url)
  # features_extracted[16] = phStatus
  # if expanded is not None:
  #   if len(expanded) >= len(url):
  #     url = expanded

  # if last_url is not None:
  #   if len(last_url) > len(url):
  #     url = last_url
  # print(url)
  features_extracted[0] = to_find_having_ip_add(url)
  features_extracted[1] = to_find_url_len(url)
  phStatus, expanded    = check_for_shortened_url(url)
  features_extracted[2] = phStatus
  features_extracted[3]  = to_find_at(url)
  features_extracted[4] = to_find_redirect(url)
  features_extracted[5] = to_find_prefix(url)
  features_extracted[6] = to_find_multi_domains(url)
  features_extracted[7] = to_find_authority(url)
  features_extracted[8] = dregisterlen(url)
  # features_extracted[9] = check_favicon(url)
  features_extracted[10] = existenceoftoken(url)
  features_extracted[11] = check_request_URL(url)
  try:
    features_extracted[12] =  check_URL_of_anchor(url)
  except:
    features_extracted[12] = '2' 

  features_extracted[13] = tags(url)
  features_extracted[14] = sfh(url)
  features_extracted[15] = check_submit_to_email(url)
  phStatus, last_url     = redirect(url)
  features_extracted[16] = phStatus
  features_extracted[17] = check_onmouseover(url)
  features_extracted[18] = check_rightclick(url)
  features_extracted[19] = check_iframe(url)
  features_extracted[20] = check_age_of_domain(url)
  features_extracted[21] = check_dns_record(url)
  features_extracted[22] = check_web_traffic(url)
  features_extracted[23] = get_pagerank(url)
  features_extracted[24] = task(url)
  features_extracted[25] = check_statistical_report(url)
  

  return features_extracted

In [ ]:
valid_url = "https://cse.iiitd.ac.in/"

In [ ]:
phish_url = "https://xavier-net.gq/?login=do"

In [ ]:
valid_url2 = "https://bit.ly/segfault"

In [ ]:
phish_url2 = "http://125.98.3.123/fake.html"

In [ ]:
def convertEncodingToPositive(data):
  mapping = {-1: 0, 1: 1}
  i = 0
  for col in data:
    data[i] = mapping[col]
    i+=1
  return data

In [ ]:
features_extracted = []

In [ ]:
import pandas as pd

In [ ]:
filename = r"C:\Users\asodh\URL_Small.csv"

In [ ]:
URL = pd.read_csv(filename)

In [ ]:
URL

In [ ]:
features_extracted_r = []

In [ ]:
for col in  URL.index :
  # print(col)
  # print(URL['URL'][col])
  if col >=1:
    web = URL['URL'][col]
    # print(web)
    # remove_last_underscore(web)
    # print(web)
    web = URL['URL'][col]
    # check_URL(web)
    # print(check_URL(web))
    try:
      r = requests.get(web) 
    except:
      r = 'Invalid'
    if r != 'Invalid': 
      features_extracted_r = extract_features(web)
      features_extracted_r[26] = URL['Label'][col]
      features_extracted.append(features_extracted_r) 

In [ ]:
web

In [ ]:
   # features_extracted_r = extract_features(valid_url2)

In [ ]:
# features_extracted_r

In [ ]:
# features_extracted.append(features_extracted_r)

In [ ]:
features_extracted

In [ ]:
 print(type(features_extracted)) 

In [ ]:
df = pd.DataFrame(features_extracted, columns =['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favion', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Redirect', 'on_mouseover', 'RightClick',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'link', 'Statistical_report', 'Label']) 
print(df)

In [ ]:
df.drop_duplicates(keep=False,inplace=True)

In [ ]:
print(df)

In [ ]:
# import pandas as pd
# cities = pd.DataFrame([['Sacramento', 'California'], ['Miami', 'Florida']], columns=['City', 'State'])
df.to_csv('Data_extracted.csv')

In [ ]:
# features_extracted = convertEncodingToPositive(features_extracted)

In [ ]:
# features_extracted

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

In [ ]:
# import numpy as np

In [ ]:
# import pickle

In [ ]:
# one_hot_enc = pickle.load(open("/One_Hot_Encoder", "rb"))

In [ ]:
# transformed_point = one_hot_enc.transform(np.array(features_extracted).reshape(1, -1))

In [ ]:
# transformed_point

In [ ]:
# model = pickle.load(open("/SVM_Final_Model", "rb"))

In [ ]:
# model.predict(transformed_point)

In [ ]:
# import sklearn

In [ ]:
# sklearn_version = sklearn.__version__

print(sklearn_version)